### Data and setup
- Load cleaned REF datasets via shared helpers.
- Prepare institutional/UoA aggregates for summaries and tests.


In [1]:

from pathlib import Path
import importlib
import sys
import pandas as pd

# Ensure local src/ is importable when run from the notebook
THIS_DIR = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
SRC_DIR = THIS_DIR if (THIS_DIR / 'statistics_summary.py').exists() else THIS_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

import statistics_helpers
importlib.reload(statistics_helpers)
from statistics_helpers import (
    load_statistics_data,
    build_descriptive_summary,
    build_inference_summary,
    llm_female_share_tables,
    build_and_save_summary_tables,
)

pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 200)

# Load raw and aggregated tables used across the summaries
df_output, df_ics, df_uoa_m, df_uni_m, df_uniuoa_m = load_statistics_data()


In [2]:
# Author-coverage share relative to the raw ICS submission count
data_root = (SRC_DIR / '..' / 'data').resolve()
raw_ics_path = '../data/final/enhanced_ref_data.csv'
raw_ics_total = len(pd.read_csv(raw_ics_path))
ics_with_authors = (df_ics[['number_male', 'number_female', 'number_unknown']].fillna(0).sum(axis=1) > 0).sum()
pct_with_authors = 100 * ics_with_authors / raw_ics_total if raw_ics_total else float('nan')

print(f"Raw ICS submissions: {raw_ics_total:,}")
print(f"ICS with author gender data: {ics_with_authors:,} ({pct_with_authors:.1f}% of raw ICS)")
print(f"ICS without author data: {raw_ics_total - ics_with_authors:,}")


Raw ICS submissions: 6,361
ICS with author gender data: 6,310 (99.2% of raw ICS)
ICS without author data: 51


In [3]:
# Author-coverage share relative to the raw REF outputs (Excel source)
raw_outputs_path = '../data/raw/raw_ref_outputs_data.xlsx'
raw_outputs_df = pd.read_excel(raw_outputs_path)
raw_output_total = len(raw_outputs_df)
outputs_with_authors = (df_output[['number_male', 'number_female', 'number_unknown']].fillna(0).sum(axis=1) > 0).sum()
pct_output_with_authors = 100 * outputs_with_authors / raw_output_total if raw_output_total else float('nan')

print(f"Raw REF outputs: {raw_output_total:,}")
print(f"Outputs with author gender data: {outputs_with_authors:,} ({pct_output_with_authors:.1f}% of raw outputs)")
print(f"Outputs without author data: {raw_output_total - outputs_with_authors:,}")


Raw REF outputs: 185,290
Outputs with author gender data: 136,016 (73.4% of raw outputs)
Outputs without author data: 49,274


In [4]:
# Female percentages by llm_* topics (ICS only)
llm_overall, llm_by_panel = llm_female_share_tables(df_ics)

print("Overall llm_* female shares (percent):")
display(llm_overall.assign(
    share_of_ics=lambda d: (d['share_of_ics'] * 100).round(2),
    pct_female=lambda d: (d['pct_female'] * 100).round(2),
))

print("\nllm_* female shares by REF panel (percent):")
display(llm_by_panel.assign(
    share_of_ics=lambda d: (d['share_of_ics'] * 100).round(2),
    pct_female=lambda d: (d['pct_female'] * 100).round(2),
))

Overall llm_* female shares (percent):


,llm_topic,n_cases,share_of_ics,female,total_people,pct_female
0,Charity,1649,25.92,2124,4325,49.11
1,Drug Trial,454,7.14,502,1564,32.10
2,Heritage,982,15.44,867,1966,44.10
3,Legislation,1918,30.15,1978,4691,42.17
4,Manufacturing,916,14.40,591,2726,21.68
5,Museum,904,14.21,854,1839,46.44
6,Nhs,1230,19.34,1815,4004,45.33
7,Patent,374,5.88,210,1187,17.69
8,School,1600,25.15,1807,3935,45.92
9,Software,1690,26.57,1641,5060,32.43



llm_* female shares by REF panel (percent):


,llm_topic,n_cases,share_of_ics,female,total_people,pct_female,panel
0,Charity,435,30.66,799,1515,52.74,A
1,Drug Trial,347,24.45,431,1243,34.67,A
2,Heritage,30,2.11,21,66,31.82,A
3,Legislation,384,27.06,553,1168,47.35,A
4,Manufacturing,221,15.57,239,690,34.64,A
5,Museum,22,1.55,20,52,38.46,A
6,Nhs,730,51.44,1286,2651,48.51,A
7,Patent,122,8.60,104,370,28.11,A
8,School,196,13.81,284,539,52.69,A
9,Software,357,25.16,614,1278,48.04,A


### Descriptive statistics
High-level counts and top/bottom breakdowns for ICS vs Outputs, including panel-level and LLM topic summaries.


In [5]:

report = build_descriptive_summary(df_ics, df_uoa_m, df_uni_m, df_output)
print(report)


DESCRIPTIVE SUMMARY OF FEMALE REPRESENTATION IN ICS & OUTPUTS

Overall female share:
  • Outputs: 295,846.0 women / 854,366.0 total = 34.63% female
  • ICS:     6,274 women / 16,447 total = 38.15% female

All-female ICS submissions (excluding unknowns): 1,298 (20.41% of all ICS cases)

LLM-tagged ICS cases (aggregate across all panels):
    llm_topic  n_cases % of ICS  female  total_people % female
      Charity     1649    25.92    2124          4325    49.11
   Drug Trial      454     7.14     502          1564    32.10
     Heritage      982    15.44     867          1966    44.10
  Legislation     1918    30.15    1978          4691    42.17
Manufacturing      916    14.40     591          2726    21.68
       Museum      904    14.21     854          1839    46.44
          Nhs     1230    19.34    1815          4004    45.33
       Patent      374     5.88     210          1187    17.69
       School     1600    25.15    1807          3935    45.92
     Software     1690    26.57

### Inference: ICS vs Outputs
One-sided tests asking whether female share in ICS exceeds Outputs at multiple levels.


In [6]:

inference_report = build_inference_summary(df_ics, df_output, df_uoa_m, df_uni_m, df_uniuoa_m)
print(inference_report)


Hypothesis across all levels: female proportion in ICS exceeds Outputs (one-sided tests).

RAW pooled female shares:
  ICS   : p̂ = 0.3815  (95% CI [0.3740, 0.3889]), n = 16447
  Output: p̂ = 0.3463  (95% CI [0.3453, 0.3473]), n = 854366

Two-proportion z-test (RAW):
  H0: p_ICS = p_Output   vs   H1: p_ICS > p_Output
  z = 9.392, p = 0 (H0: p1 = p2 vs H1: p1 > p2). Observed difference p1−p2 = +0.0352 [+0.0277, +0.0427] (95% CI, Wald, unpooled). Result is statistically significant at α=0.05; the estimated difference is positive.
  Interpretation: This tests the overall female share across all observations. A significant result supports higher ICS share.


— University level: paired analysis for Δ = (ICS − Output)
  Descriptives: n = 155, mean(Δ) = 0.0105 (95% CI [-0.0096, 0.0305]), sd = 0.1273, Cohen's dz = 0.082
  t-test (mean Δ > 0): t = 1.023, p = 0.154 → not significant at α=0.05 (mean Δ positive).
  Wilcoxon (median Δ > 0): W = 7525.000, p = 0.0041 → significant (median Δ positive)

### LaTeX summary tables
Save panel, UoA, and LLM topic tables; print them and note the output path.


In [7]:
# Build and save LaTeX tables
tables_out_dir = Path('../outputs/tables')
tables = build_and_save_summary_tables(df_ics, df_output, out_dir=tables_out_dir)
print(f'Saved LaTeX tables to: {tables_out_dir.resolve()}')
print('Files:')
for p in sorted(tables_out_dir.glob('*.tex')):
    print(' -', p.name)

print('Panel summary (ICS vs Outputs):', display(tables['panel']))
 
print('UoA summary (ICS vs Outputs):', display(tables['uoa']))

print('LLM topics (aggregate, ICS only):', display(tables['llm']))

print('LLM topics by Panel (ICS only):', display(tables['llm_panel']))

Saved LaTeX tables to: /home/porco/Dropbox/ics_work/ref_gender/outputs/tables
Files:
 - llm_panel_summary.tex
 - llm_summary.tex
 - panel_summary.tex
 - regression_results.tex
 - uoa_summary.tex


,Panel,FTE,PhDs (000),Total Income (£bn),N (ICS),% Female (ICS),% All Female (ICS),N (Papers),% Female (Papers),% All Female (Papers)
0,A,149915,366,232.700,1419,45.49,34.05,41253,39.83,67.240
1,B,98047,311,104.800,1268,18.08,10.87,33953,19.13,12.530
2,C,110106,174,17.700,2146,41.70,34.57,44770,35.88,16.760
3,D,54291,88,5.305,1528,46.34,20.51,16040,44.62,3.466


Panel summary (ICS vs Outputs): None


,UoA,Unit of Assessment,Panel,FTE,PhDs (000),Total Income (£bn),N (ICS),% Female (ICS),% All Female (ICS),N (Papers),% Female (Papers),% All Female (Papers)
0,1,Clinical Medicine,A,53711,134,127.8000,254,30.61,4.3510,9309,38.880,23.06000
1,2,"Public Health, Health Services and Primary Care",A,15662,20,26.0800,151,47.70,5.6100,3525,41.820,9.36700
2,3,"Allied Health Professions, Dentistry, Nursing ...",A,28252,60,12.1300,393,56.96,12.3800,10093,43.620,12.85000
3,4,"Psychology, Psychiatry and Neuroscience",A,25203,71,28.5600,326,54.50,7.8260,8609,41.030,10.98000
4,5,Biological Sciences,A,18968,64,30.7900,192,27.66,2.1520,6520,35.300,7.79500
5,6,"Agriculture, Food and Veterinary Sciences",A,8119,18,7.3050,103,34.50,1.7210,3197,36.310,3.19800
6,7,Earth Systems and Environmental Sciences,B,8510,20,6.9560,148,26.10,2.0720,3816,27.870,3.21100
7,8,Chemistry,B,4834,25,6.8940,113,18.58,0.8766,3350,25.140,2.03600
8,9,Physics,B,11330,34,20.8800,169,14.36,1.3710,3481,9.068,1.61500
9,10,Mathematical Sciences,B,10814,21,3.4510,176,23.81,1.5940,4302,19.480,0.73590


UoA summary (ICS vs Outputs): None


,Topic,Number of ICS,% Female Authors,% of All Female Authors
0,Charity,1649,49.109827,33.854001
1,Startup,801,23.657237,9.196685
2,Patent,374,17.691660,3.347147
3,Museum,904,46.438282,13.611731
4,Nhs,1230,45.329670,28.928913
5,Drug Trial,454,32.097187,8.001275
6,School,1600,45.921220,28.801403
7,Legislation,1918,42.165849,31.526937
8,Heritage,982,44.099695,13.818935
9,Manufacturing,916,21.680117,9.419828


LLM topics (aggregate, ICS only): None


,Panel,Topic,Number of ICS,% Female Authors,% of All Female Authors (panel)
0,A,Charity,435,52.739274,37.406367
1,A,Startup,181,33.612040,9.410112
2,A,Patent,122,28.108108,4.868914
3,A,Museum,22,38.461538,0.936330
4,A,Nhs,730,48.509996,60.205993
5,A,Drug Trial,347,34.674175,20.177903
6,A,School,196,52.690167,13.295880
7,A,Legislation,384,47.345890,25.889513
8,A,Heritage,30,31.818182,0.983146
9,A,Manufacturing,221,34.637681,11.189139


LLM topics by Panel (ICS only): None
